In [ ]:
import itertools
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point, MultiPoint
from shapely.affinity import translate, rotate, scale
from functools import partial
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')


def create_polygons(center_blocks):
    """
    Crea poligonos con la clase shapely de python usando los puntos centrales
    donde estos puntos centrales consisten en una coleccion de tuplas(x, y ,m) donde 
    x e y son coordenadas del tablero, y la m es un valor entre 0 y 1
    :param coordenadas_tablero: coleccion de puntos del tablero
    :return: arreglo con los poligonos iniciales
    """
    polygons = []
    for idx, block in enumerate(center_blocks):
        pts = np.array(block)[:, :-1]
        poly = MultiPoint(pts+0.5).buffer(0.5, cap_style=3)
        poly.checkers = np.array(block)
        poly.orientations = get_unique_orientations(poly)
        poly.id = idx
        polygons.append(poly)
    return np.array(polygons)


def get_unique_orientations(poly):
    """
    Obtiene las posibles rotaciones y caras de una pieza, toma en cuenta la simetria
    y obtiene las orientaciones unicas para el poligono dado
    :param poligono: poligono a operar
    :return: arreglo de orientaciones unicas
    """
    cx, cy = poly.centroid.x, poly.centroid.y
    pts = np.array(poly.checkers)[:, :-1]
    checkers = np.array(poly.checkers)[:, -1]
    pts = pts[checkers == 1]
    poly = poly.difference(MultiPoint(pts + 0.5).buffer(0.2))

    rots = [0, 90, 180, 270]
    flips = [1, -1]
    iterables = [rots, flips]
    settings = np.array(list(itertools.product(*iterables)))
    unique_polys = []
    unique_settings = []
    for rot, flip in settings:
        p = poly
        p = rotate(p, rot, origin=[cx, cy])
        p = scale(p, flip, origin=[cx, cy, 0])
        if np.any([p.difference(u).area < 1e-5 for u in unique_polys]):
            continue
        unique_polys.append(p)
        unique_settings.append([rot, flip])
    return unique_settings


def transform(poly, vec, r, f):
    """
    Aplica una transformacion al poligono
    :param poligono: Poligono a ser transformado
    :param vector_t: Vector de traslacion
    :param angulo_r: angulo de rotacion en grados
    :param cara_f: cara de la figura (1,-1)
    :return: poligono transformado
    """
    poly = rotate(poly, r, origin=np.array([0.5, 0.5]))
    poly = scale(poly, f, origin=np.array([0.5, 0.5, 0.0]))
    poly = translate(poly, *vec)
    return poly


def calculate_outline(profile):
    """
    Funcion que contiene el calculo del perimetro de un nuevo perfil
    :param perfil: perfil inicial
    :param poligono: poligono a colocar
    :param configuracion: configuraciones para transformar el poligono
    :return: perimetro del nuevo perfil
    """
    if profile is None:
        return np.nan

    outline = profile.exterior.length
    for interior in profile.interiors:
        outline += interior.length
    return outline


def get_new_profile(profile, poly, setting):
    """
    Funcion para encontrar la colocacion optima de un poligono en un perfil. Si el poligono no puede ser colocado
    en el perfil dado, regresa None, de otra forma se retorna la configuracion optima y el perfil correspondiente.
    :param perfil: perfil de entrada
    :param poligono: poligono a verificar su colocacion optima
    :param tamanio_tablero: tamaño del tablero
    :param origin_checker: checador del origen
    :return: configuracion de la colocacion optima y el perfil correspondiente
    """
    poly = transform(poly, *setting)
    if not poly.within(profile):
        return None

    new_profile = profile.difference(poly)
    if type(new_profile) == MultiPolygon:
        return None

    return new_profile


def get_new_outline(profile, poly, setting):
    """
    Funcion que contiene el calculo del perimetro de un nuevo perfil
    :param perfil: perfil inicial
    :param poligono: poligono a colocar
    :param configuracion: configuraciones para transformar el poligono
    :return: perimetro del nuevo perfil
    """
    return calculate_outline(get_new_profile(profile, poly, setting))


def optimal_placement(profile, poly, board_size, origin_checker=None):
    """
    Funcion para encontrar la colocacion optima de un poligono en un perfil. Si el poligono no puede ser colocado
    en el perfil dado, regresa None, de otra forma se retorna la configuracion optima y el perfil correspondiente.
    :param perfil: perfil de entrada
    :param poligono: poligono a verificar su colocacion optima
    :param tamanio_tablero: tamaño del tablero
    :param origin_checker: checador del origen
    :return: configuracion de la colocacion optima y el perfil correspondiente
    """
    point_grid = np.array([Point([x + 0.5, y + 0.5]) for x in range(board_size) for y in range(board_size)],
                          dtype=object)
    position_grid = np.array([[x, y] for x in range(board_size) for y in range(board_size)], dtype=int)
    point_mask = np.array([pt.within(profile) for pt in point_grid], dtype=bool)

    if origin_checker is not None:
        if origin_checker == poly.checkers[0, -1]:
            point_mask &= np.sum(position_grid, axis=-1) % 2 == 0
        else:
            point_mask &= np.sum(position_grid, axis=-1) % 2 == 1

    rflips = poly.orientations
    pos = position_grid[point_mask]

    iterables = [pos, rflips]
    settings = list(itertools.product(*iterables))
    settings = [(s[0], *s[1]) for s in settings]

    func = partial(get_new_outline, profile, poly)

    pool = Pool(processes=None)
    outlines = np.array(pool.map(func, settings))
    pool.close()

    outlines = outlines.astype(float)
    opt_setting = settings[np.argsort(outlines)[0]]
    opt_profile = get_new_profile(profile, poly, opt_setting)

    if opt_profile is not None:
        return opt_setting, opt_profile


def cantor(a, b):
    """
    Funcion de cantor usada para obtener numeros unicos de un par de numeros
    :param a: numero de entrada
    :param b: otro numero de entrada
    :return: numero operado
    """
    return 0.5 * (a + b + 1) * (a + b) + b


def get_optimal_configuration(chromosome, initial_polygons, board_size):
    """
    Put each piece in the chromosome order in the optimal place until the final configuration is
    achieved. return the fitness level and the corresponding placements.
    Coloca cada pieza en el orden optimo dentro del cromosoma hasta que se llena todo el cromosoma
    retornando la aptitud y los elementos correspondientes
    :param cromosoma: cromosoma a ser llenado
    :param poligonos iniciales: lista de poligonos inicial
    :param tamanio_tablero: tamaño del tablero
    :return: colocaciones y aptitud
    """
    profile = Polygon([[0, 0], [board_size, 0], [board_size, board_size], [0, board_size]])
    placements = np.full_like(chromosome, None, dtype=object)
    origin_checker = None
    for idx, c in enumerate(chromosome):
        p = initial_polygons[c]
        try:
            opt, profile = optimal_placement(profile, p, board_size, origin_checker)
            placements[idx] = opt
            if origin_checker is None:
                sum_vec = np.sum(opt[0])
                if sum_vec % 2 == p.checkers[0, -1]:
                    origin_checker = 0
                else:
                    origin_checker = 1
        except Exception:
            idx -= 1
            break
    empty_area = profile.area
    n_unused_pieces = len(chromosome) - idx - 1
    outline = calculate_outline(profile)
    fitness_score = empty_area + n_unused_pieces + outline
    return placements, fitness_score

In [ ]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import MultiPoint, Polygon

def plot_chromosome(individual, initial_polygons, board_size, filename=''):
    """
    Funcion para graficar el tablero con las piezas ensambladas
    :param individuo: El individuo que contiene todos los atributos como la funcion de aptitud
    :param poligonos: lista de poligonos
    :param tamanio_tablero: tamanio de nuestro tablero
    :param nom_archivo: nombre del fichero a guardar
    """
    new_polys = []
    for poly in initial_polygons:
        pts = np.array(poly.checkers)[:, :-1]
        checkers = np.array(poly.checkers)[:, -1]
        pts = pts[checkers == 1]
        poly = poly.difference(MultiPoint(pts + 0.5).buffer(0.25))
        new_polys.append(poly)

    new_polys = np.array(new_polys)
    new_polys = [transform(p, *s) for p, s in zip(new_polys[individual.chromosome],
                                                  individual.placements) if s is not None]
    indices = [individual.chromosome[idx] for idx, s in enumerate(individual.placements) if s is not None]
    canvas = Polygon([[0, 0], [board_size, 0], [board_size, board_size], [0, board_size]])
    gdf = gpd.GeoDataFrame({'idx': indices}, geometry=new_polys)

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.plot(*canvas.exterior.xy, 'k')
    gdf.plot(ax=ax, column='idx', edgecolor='black', alpha=0.75, vmin=0, vmax=len(initial_polygons), cmap=plt.cm.hot)
    ax.set_ylim([-0.1, board_size + 0.1])
    ax.set_xlim([-0.1, board_size + 0.1])
    ax.set_title(f'[{",".join(individual.chromosome.astype(str))}]', fontsize=20, fontname='serif')
    ax.set_xlabel(f'Aptitud: {int(individual.fitness)}', fontsize=20, fontname='serif')
    ax.set_xticks([])
    ax.set_yticks([])

    if filename != '':
        fig.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()


def plot_history(history, filename=''):
    """
    Funcion para graficar el proceso de evolucion
    :param historial: datos para cada generacion (historial)
    :param nom_archivo: nombre del archivo a guardar 
    """
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    for idx, gen in enumerate(history):
        ax.scatter(np.full_like(gen, idx), gen, s=50, c='k', marker='o')
        ax.set_title('Proceso de evolucion', fontsize=20, fontname='serif')
        ax.set_xlabel('Numero de generacion', fontsize=20, fontname='serif')
        ax.set_ylabel('Aptitud', fontsize=20, fontname='serif')
        ax.set_ylim(bottom=0.0)

    plt.plot(range(len(history)), np.mean(history, axis=1), label='Mean generation value')
    plt.fill_between(range(len(history)), np.mean(history, axis=1) - np.std(history, axis=1),
                     np.mean(history, axis=1) + np.std(history, axis=1),
                     alpha=0.5, interpolate=True)
    plt.legend(prop={'family': 'serif', 'size': 15})

    if filename != '':
        fig.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

class Individual(object):
    """
    Esta clase representa al individuo
    """
    def __init__(self, polygons, board_size, chromosome=None):
        """
        Un nuevo cromosoma es generado si no se especifica uno
        :param poligonos: lista de poligonos
        :param tamanio_tablero: tamaño del tablero
        :param cromosoma: establece el orden en que se colocaran los poligonos
        """
        self.n_pieces = len(polygons)
        self.chromosome = self.generate_chromosome() if chromosome is None else np.array(chromosome)
        self.polygons = polygons
        self.board_size = board_size
        self.placements, self.fitness = get_optimal_configuration(self.chromosome, polygons, board_size)

    def generate_chromosome(self):
        """
        Crea una secuencia aleatoria de numeros para el cromosoma
        """
        return np.random.choice(range(self.n_pieces), self.n_pieces, replace=False).astype(int)

    def mutate(self, mutation_probability):
        """
        Muta al cromosoma intercambiando el gen con un gen aleatorio si se cumple la condicion de cruza
        :param probabilidad_mutacion: probabilidad de que la mutacion ocurra
        """
        chromosome = self.chromosome.copy()
        has_mutated = False
        for idx, c in enumerate(chromosome):
            if np.random.rand() < mutation_probability:
                has_mutated = True
                rand_idx = np.random.choice(range(len(chromosome)))
                chromosome[idx], chromosome[rand_idx] = chromosome[rand_idx], chromosome[idx]

        if has_mutated:
            self.chromosome = chromosome
            self.placements, self.fitness = get_optimal_configuration(self.chromosome, self.polygons, self.board_size)

    def mate(self, partner):
        """
        Establecemos la cruza por medio de ox
        :param acompanante: el segundo individuo con el que se cruzara
        :return: dos nuevos hijos que son resultado de la cruza
        """
        c1, c2 = np.zeros((2, self.n_pieces), dtype=int)
        start, end = np.sort(np.random.choice(range(self.n_pieces), 2, replace=False))

        p1 = self.chromosome
        p2 = partner.chromosome

        c1[start:end + 1] = p1[start:end + 1]
        mask1 = ~np.in1d(p1, p1[start:end + 1])
        mask2 = ~np.in1d(p2, p1[start:end + 1])
        c1[mask1] = p2[mask2]

        c2[start:end + 1] = p2[start:end + 1]
        mask1 = ~np.in1d(p2, p2[start:end + 1])
        mask2 = ~np.in1d(p1, p2[start:end + 1])
        c2[mask1] = p1[mask2]

        return Individual(self.polygons, self.board_size, c1), Individual(self.polygons, self.board_size, c2)

    def plot(self, filename=''):
        """
        Grafica el tablero con las piezas colocadas
        :param nom_archivo: nombre del archivo donde se guardara la imagen
        """
        if filename == '':
            plot_chromosome(self, self.polygons, self.board_size)
        else:
            plot_chromosome(self, self.polygons, self.board_size, filename)


class Evolution(object):
    """
    Clase que representa la evolucion de la poblacion
    """
    def __init__(self, n_population, polygons, board_size, mutation_probability=0.01, generations=10):
        """
        Funcion que inicializa la poblacion, un cromosoma aleatorio es generado si no se especifica uno
        :param num_poblacion: tamanio de la poblacion para trabajar
        :param polygons: lista de figuras inicial
        :param board_size: tamanio del tablero
        :param mutation_probability: probabilidad de que ocurra una mutacion
        """
        self.n_population = n_population + 1 if n_population % 2 != 0 else n_population
        self.polygons = polygons
        self.board_size = board_size
        self.mutation_probability = mutation_probability
        self.generations = generations
        self.population = None
        self.best = None
        self.history = []

    def initialize_population(self):
        """
        inicializamos una nueva poblacion
        :return: una lista de individuos
        """
        print(f'Generacion: {len(self.history)}: Inicializando')
        self.population = [Individual(self.polygons, self.board_size) for _ in tqdm(range(self.n_population),
                                                                                    leave=False)]
        self.mutate_generation()

    def select_best_pair(self):
        """
        Selecciona dos individuos de la poblacion aleatoriamente, donde la probabilidad de elegir un individuo
        esta ponderada por su nivel de aptitud.
        :return: 2 individuos seleccionados
        """
        pop_fitness_list = np.array([ind.fitness for ind in self.population])
        pop_fitness_list = -pop_fitness_list + np.min(pop_fitness_list) + np.max(pop_fitness_list) + 1
        pop_fitness_list = pop_fitness_list / np.sum(pop_fitness_list)
        p1, p2 = np.random.choice(self.population,
                                  size=2,
                                  replace=False,
                                  p=pop_fitness_list)
        return p1, p2

    def mutate_generation(self):
        """
        Aplica una ronda de mutacion a la poblacion
        """
        for ind in self.population:
            ind.mutate(self.mutation_probability)

    def next_generation(self):
        """
        Crea una nueva generacion de descendientes
        """
        print(f'Generacion: {len(self.history)}')
        new_population = []
        for _ in tqdm(range(int(self.n_population/2)), leave=False):
            p1, p2 = self.select_best_pair()
            c1, c2 = p1.mate(p2)
            new_population.extend([c1, c2])
        self.population = new_population
        self.mutate_generation()

    def check_condition(self):
        """
        Checa si se ha alcanzado el numero esperado de generaciones
        :return: boolean
        """
        pop_fitness_list = np.array([ind.fitness for ind in self.population], dtype=int)
        if np.any(pop_fitness_list == 0):
            return True

    def get_best_candidate(self):
        """
        Obtiene el mejor individuo de la poblacion actual
        :return: el mejor individuo
        """
        best_id = np.argsort([ind.fitness for ind in self.population])[0]
        return self.population[best_id]

    def plot_process(self, filename=''):
        """
        Grafica el proceso de evolucion de la poblacion
        :param filename: filename for saving the image
        """
        if filename == '':
            plot_history(np.array(self.history))
        else:
            plot_history(np.array(self.history), filename)

    def check_generations(self, actual):
        """
        Verificamos que no se haya llegado al tope de generaciones
        :param limit: limite establecido por el usuario
        :param actual: numero de generaciones actuales
        """
        limit = self.generations
        if actual>=limit:
          return True
        else:
          return False

    def run(self, filename=''):
        """
        Ejecuta el proceso de evolucion
        :param nom_archivo: nombre del archivo
        """
        self.initialize_population()
        self.best = self.get_best_candidate()
        pop_fitness_list = np.array([ind.fitness for ind in self.population], dtype=int)
        self.history.append(pop_fitness_list)
        self.plot_process(filename)
        plt.close()
        print(f'Aptitud del mejor candidato: {self.best.fitness}, '
              f'        cromosoma: [{",".join(np.array(self.best.chromosome, dtype=str))}]')
        while not self.check_condition() and not self.check_generations(len(self.history)):
            self.next_generation()
            best = self.get_best_candidate()
            pop_fitness_list = np.array([ind.fitness for ind in self.population], dtype=int)
            self.history.append(pop_fitness_list)
            self.plot_process(filename)
            plt.close()
            if best.fitness < self.best.fitness:
                self.best = best
            print(f'Aptitud del mejor candidato: {self.best.fitness}, '
                  f'cromosoma: [{",".join(np.array(self.best.chromosome, dtype=str))}]')
        
        if(self.check_condition()):
          print('------------ Resultados ---------------')
          print('Se ha concluido de armar el puzzle con exito')
          print(f'Solucion (cromosoma): [{",".join(np.array(self.best.chromosome, dtype=str))}]')
          self.best.plot('./Solucion')
          print('Tablero guardado')
        else:
          print('------------ Resultados ---------------')
          print('Sin solucion optima')
          print(f'Candidato con la mejor aptitud:  {self.best.fitness}')
          print(f'Numero de generaciones: {len(self.history)}')
          self.best.plot('./Armado_tablero')
          print('Tablero guardado')

